In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
root = '/gdrive/My Drive/MSB481/'

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Model

In [4]:
train_dir = os.path.join(root, "Garbage classification/")
labels = ['paper', 'plastic']

In [5]:
class_model = tf.keras.models.load_model(os.path.join(root, 'downloaded_model.h5'))
class_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        80        
                                                                 
 activation (Activation)     (None, 96, 96, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        8

In [8]:
own_model = Sequential()
own_model.add(Conv2D(16, (1, 1), input_shape=(96, 96, 3)))
own_model.add(Activation('relu'))
own_model.add(MaxPooling2D(pool_size=(2, 2)))

own_model.add(Conv2D(32, (1, 1)))
own_model.add(Activation('relu'))
own_model.add(MaxPooling2D(pool_size=(2, 2)))

own_model.add(Conv2D(64, (1, 1)))
own_model.add(Activation('relu'))
own_model.add(MaxPooling2D(pool_size=(2, 2)))

own_model.add(Conv2D(128, (1, 1)))
own_model.add(Activation('relu'))
own_model.add(MaxPooling2D(pool_size=(2, 2)))

own_model.add(Dropout(0.5))
own_model.add(Flatten()) 
# own_model.add(Activation('sigmoid'))
own_model.add(Dropout(0.5))
own_model.add(Dense(1, activation = 'sigmoid'))

own_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        64        
                                                                 
 activation (Activation)     (None, 96, 96, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        544       
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                    

In [9]:
class_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate = 0.0001), metrics=['accuracy'])

In [10]:
own_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate = 0.0001), metrics=['accuracy'])

In [11]:
# creating generators for combining data and increasing the gainable insights by slightly modifying the images in the dataset

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                                   rotation_range=15,zoom_range=0.1,
                                   width_shift_range=0.15,height_shift_range=0.15,
                                   shear_range=0.1,
                                   fill_mode="nearest",
                                   rescale=1./255., 
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(96, 96), batch_size=32, class_mode='binary', subset='training', classes = ['paper', 'plastic'])
validation_generator = train_datagen.flow_from_directory(train_dir, target_size=(96, 96), batch_size=32, class_mode='binary', subset='validation', classes = ['paper', 'plastic'])

Found 870 images belonging to 2 classes.
Found 217 images belonging to 2 classes.


In [12]:
history = own_model.fit(train_generator, epochs=150, verbose=1, validation_data=validation_generator)

Epoch 1/150
28/28 [==============================] - 10s 225ms/step - loss: 0.6902 - accuracy: 0.5195 - val_loss: 0.6891 - val_accuracy: 0.5484
Epoch 2/150
28/28 [==============================] - 6s 215ms/step - loss: 0.6917 - accuracy: 0.5126 - val_loss: 0.6861 - val_accuracy: 0.5484
Epoch 3/150
28/28 [==============================] - 6s 216ms/step - loss: 0.6860 - accuracy: 0.5529 - val_loss: 0.6855 - val_accuracy: 0.5484
Epoch 4/150
28/28 [==============================] - 6s 215ms/step - loss: 0.6813 - accuracy: 0.5552 - val_loss: 0.6851 - val_accuracy: 0.5484
Epoch 5/150
28/28 [==============================] - 6s 217ms/step - loss: 0.6837 - accuracy: 0.5494 - val_loss: 0.6855 - val_accuracy: 0.5484
Epoch 6/150
28/28 [==============================] - 6s 215ms/step - loss: 0.6823 - accuracy: 0.5448 - val_loss: 0.6849 - val_accuracy: 0.5484
Epoch 7/150
28/28 [==============================] - 6s 217ms/step - loss: 0.6816 - accuracy: 0.5460 - val_loss: 0.6839 - val_accuracy: 0.548

In [13]:
own_model.save_weights(os.path.join(root, "model/best_cnn_binary_small.h5"))